In [22]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier

#Cleaning data by removing the useless 
df = pd.read_csv("/mnt/d/projects/house prices/Egypt_Houses_Price.csv/Egypt_Houses_Price.csv")
df = df.apply(lambda col: col.str.strip().str.lower() if col.dtype == "object" else col)
df.replace("unknown", None, inplace=True)
df["Type"] = df["Type"].replace("standalone villa","stand alone villa")
df["Level"] = df["Level"].replace("ground", "0")
df["Level"] = df["Level"].replace("10+", "11")
df = df[df["Level"] != "highest"]
complete_data = df.dropna()
df = df[df.isnull().sum(axis=1) <= 1]

#predicting missing "furnished" values
features = ["Compound", "Price", "Type", "Area"]
target = "Furnished"
x_furnished_train = complete_data[features]
y_furnished_train = complete_data[target]
x_furnished_train = pd.get_dummies(x_furnished_train)
clf = DecisionTreeClassifier()
clf.fit(x_furnished_train, y_furnished_train)

to_predict = df[df["Furnished"].isnull()].copy()
x_furnished_test = to_predict[features]
x_furnished_test = pd.get_dummies(x_furnished_test)
x_furnished_test = x_furnished_test.reindex(columns=x_furnished_train.columns, fill_value=0)
predictions = clf.predict(x_furnished_test)
df.loc[to_predict.index, "Furnished"] = predictions
complete_data = pd.concat([complete_data, df.loc[to_predict.index]])

#predicting missing "level" values
target = "Level"
x_level_train = complete_data[features]
y_level_train = complete_data[target]
x_level_train = pd.get_dummies(x_level_train)
clf_level = DecisionTreeClassifier()
clf_level.fit(x_level_train, y_level_train)
to_predict_level = df[df["Level"].isnull()].copy()
x_level_test = to_predict_level[features]
x_level_test = pd.get_dummies(x_level_test)
x_level_test = x_level_test.reindex(columns=x_level_train.columns, fill_value=0)
level_predictions = clf_level.predict(x_level_test)
df.loc[to_predict_level.index, "Level"] = level_predictions
complete_data = pd.concat([complete_data, df.loc[to_predict_level.index]])

#save cleaned data
complete_data.to_csv("/mnt/d/projects/house prices/Egypt_Houses_Price.csv/cleaned_data.csv")


print(df.isnull().sum())
print()
print(complete_data.shape[0])
print()
print(df["Level"].value_counts(dropna=False))


Type                 0
Price                0
Bedrooms             0
Bathrooms            0
Area                10
Furnished            0
Level                0
Compound          2843
Payment_Option       0
Delivery_Date     1256
Delivery_Term      221
City                 0
dtype: int64

9578

Level
0     6377
1     2352
2     2270
3     1307
4      615
5      395
6      143
11     128
7      118
8       80
10      65
9       58
Name: count, dtype: int64
